# Event Planning Automation

##### In this project, we will automate event planning workflow using 3 Agents
- Venue Coordinator
- Logistics Manager
- Marketing and Communications Agent

##### These 3 Agents will perform the following tasks

- Venue Task (done by Venue Coordinator)
- Logistics Task (done by Logistics Manager) - **Async Task**
- Marketing Task (done by Marketing and Communications Agent) - **Async Task**

Venue Task will generate a json file `venue_details.json` using a Pydantic model `VenueDetails`

Venue Task and Logistics Task will wait for human input to asses and verify the output before finalizing it

## Step 01. Installing the libraries (crewai and crewai_tools)

**Note:** crewai requires python version >=3.10 and <=3.13

In [1]:
import sys
print(sys.version)

3.11.8 (main, Feb 26 2024, 15:43:17) [Clang 14.0.6 ]


In [2]:
# !pip install crewai

In [3]:
# !pip install crewai_tools

In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Step 02. Importing modules, LLMs and libraries 

**Note:** We need to provide openai model name, and api_keys as environment variables in order to use Agent class

**Note:** SerperDevTool (tool by crewai, for Google search) also requires api key

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_MODEL_NAME"] = "gpt-4-0125-preview"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [59]:
from crewai import Agent, Task, Crew

## Step 03. Creating Agents

Creating 3 agents
- Venue Coordinator
- Logistics Manager
- Marketing and Communications Agent

### crewAI Tools

In [60]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

### Agent 1: Venue Coordinator 

In [61]:
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True
)

### Agent 2: Logistics Manager

In [62]:
logisitics_manager = Agent(
    role="Logistics Manager",
    goal=(
        "Manage all logistics for the event "
        "including catering and equipment"
    ),
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True
)

### Agent 3: Marketing and Communication Agent 

In [63]:
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    tools=[search_tool, scrape_tool],
    verbose=True
)

## Step 03. Creating Tasks

Here, we will be creating 3 tasks

- Venue Task (done by Venue Coordinator)
- Logistics Task (done by Logistics Manager) - **Async Task**
- Marketing Task (done by Marketing and Communications Agent) - **Async Task**

### Creating VenueDetails Pydantic Model

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [64]:
from pydantic import BaseModel

In [65]:
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

### Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

#### Task 1: Venue Task

In [66]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    output_json=VenueDetails,
    output_file="venue_details.json",  # Outputs the venue details as a JSON file
    agent=venue_coordinator,
    human_input=True
    # human_input=False
)

### Task 2: Logistics Task

In [67]:
logsitics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    agent=logisitics_manager,
    human_input=True,
    # human_input=False,
    async_execution=False, ## 2 tasks cant be async
)

### Task 3: Marketing Task 

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [68]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    agent=marketing_communications_agent,
    async_execution=True,
    output_file="marketing_report.md" # Outputs the report as a text file
)

## Creating Crew 

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [69]:
crew = Crew(
    agents=[venue_coordinator, logisitics_manager, marketing_communications_agent],
    tasks=[venue_task, logsitics_task, marketing_task],
    verbose=True
)

### Running the crew

Setting the inputs for execution of the crew

In [75]:
event_details = {
    'event_topic': "Marriage Reception Ceremony",
    'event_description': "A gathering of family, close relatives and friends "
                         "to attend our Marriage Reception Ceremony.",
    'event_city': "Siliguri",
    'tentative_date': "2025-04-17",
    'expected_participants': 2000,
    'budget': 1500000,
    'venue_type': "Banquet Halls or Playgrounds"
}

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [ ]:
results = crew.kickoff(inputs=event_details)

In [72]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': '1, Udham Singh Sarani, Sevoke Road, Siliguri - 734001 (Near Ghosh '
            'Nursery Building)',
 'booking_status': 'Please contact them directly for booking information and '
                   'to confirm availability for your desired dates.',
 'capacity': 0,
 'name': 'Soubhagya Palace'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [52]:
from IPython.display import Markdown
Markdown("marketing_report.md")

Thought: To create an effective marketing plan and gauge potential attendee engagement, I need to understand current trends in promoting events such as a Marriage Reception Ceremony. This will help identify the best platforms and strategies for reaching our target audience of at least 2000 potential attendees. I should start by researching the most effective ways to market a wedding reception or similar events.